In [7]:
import re
import hashlib
import logging
import collections
from itertools import groupby

In [8]:
def _hashfunc(x):
    return int(hashlib.md5(x).hexdigest(), 16)

In [15]:
class Simhash(object):

    def __init__(self, value, f=64, reg=r'[\w\u4e00-\u9fcc]+', hashfunc=None):
        """
        `f` is the dimensions of fingerprints
        `reg` is meaningful only when `value` is basestring and describes
        what is considered to be a letter inside parsed string. Regexp
        object can also be specified (some attempt to handle any letters
        is to specify reg=re.compile(r'\w', re.UNICODE))
        `hashfunc` accepts a utf-8 encoded string and returns a unsigned
        integer in at least `f` bits.
        """

        self.f = f
        self.reg = reg
        self.value = None

        if hashfunc is None:
            self.hashfunc = _hashfunc
        else:
            self.hashfunc = hashfunc

        if isinstance(value, Simhash):
            self.value = value.value
        elif isinstance(value, basestring):
            self.build_by_text(unicode(value))
        elif isinstance(value, collections.Iterable):
            self.build_by_features(value)
        elif isinstance(value, long):
            self.value = value
        else:
            raise Exception('Bad parameter with type {}'.format(type(value)))

    def _slide(self, content, width=4):
        return [content[i:i + width] for i in range(max(len(content) - width + 1, 1))]

    def _tokenize(self, content):
        content = content.lower()
        content = ''.join(re.findall(self.reg, content))
        ans = self._slide(content)
        return ans

    def build_by_text(self, content):
        features = self._tokenize(content)
        features = {k:sum(1 for _ in g) for k, g in groupby(sorted(features))}
        return self.build_by_features(features)

    def build_by_features(self, features):
        """
        `features` might be a list of unweighted tokens (a weight of 1
                   will be assumed), a list of (token, weight) tuples or
                   a token -> weight dict.
        """
        v = [0] * self.f
        masks = [1 << i for i in range(self.f)]
        if isinstance(features, dict):
            features = features.items()
        for f in features:
            if isinstance(f, basestring):
                h = self.hashfunc(f.encode('utf-8'))
                w = 1
            else:
                assert isinstance(f, collections.Iterable)
                h = self.hashfunc(f[0].encode('utf-8'))
                w = f[1]
            for i in range(self.f):
                v[i] += w if h & masks[i] else -w
        ans = 0
        for i in range(self.f):
            if v[i] >= 0:
                ans |= masks[i]
        self.value = ans

    def distance(self, another):
        assert self.f == another.f
        x = (self.value ^ another.value) & ((1 << self.f) - 1)
        ans = 0
        while x:
            ans += 1
            x &= x - 1
        return ans


class SimhashIndex(object):

    def __init__(self, objs, f=64, k=2):
        """
        `objs` is a list of (obj_id, simhash)
        obj_id is a string, simhash is an instance of Simhash
        `f` is the same with the one for Simhash
        `k` is the tolerance
        """
        self.k = k
        self.f = f
        count = len(objs)
        logging.info('Initializing %s data.', count)

        self.bucket = collections.defaultdict(set)

        for i, q in enumerate(objs):
            if i % 10000 == 0 or i == count - 1:
                logging.info('%s/%s', i + 1, count)

            self.add(*q)

    def get_near_dups(self, simhash):
        """
        `simhash` is an instance of Simhash
        return a list of obj_id, which is in type of str
        """
        assert simhash.f == self.f

        ans = set()
        
        print [y for y in self.get_keys(simhash)]
        print self.bucket.keys()
        print '*'*40

        for key in self.get_keys(simhash):
            dups = self.bucket[key]
            logging.debug('key:%s', key)
            if len(dups) > 200:
                logging.warning('Big bucket found. key:%s, len:%s', key, len(dups))

            for dup in dups:
                sim2, obj_id = dup.split(',', 1)
                sim2 = Simhash(long(sim2, 16), self.f)

                d = simhash.distance(sim2)
                if d <= self.k:
                    ans.add(obj_id)
        return list(ans)

    def add(self, obj_id, simhash):
        """
        `obj_id` is a string
        `simhash` is an instance of Simhash
        """
        assert simhash.f == self.f

        print 'OBJ_ID = ', obj_id, '; value = ', simhash.value
        print 'KEYS => ', [e for e in self.get_keys(simhash)]
        for key in self.get_keys(simhash):
            v = '%x,%s' % (simhash.value, obj_id)
            print '\tkey => ', key, '; current set => ', self.bucket[key]
            self.bucket[key].add(v)

    def delete(self, obj_id, simhash):
        """
        `obj_id` is a string
        `simhash` is an instance of Simhash
        """
        assert simhash.f == self.f

        for key in self.get_keys(simhash):
            v = '%x,%s' % (simhash.value, obj_id)
            if v in self.bucket[key]:
                self.bucket[key].remove(v)

    @property
    def offsets(self):
        """
        You may optimize this method according to <http://www.wwwconference.org/www2007/papers/paper215.pdf>
        """
        return [self.f // (self.k + 1) * i for i in range(self.k + 1)]

    def get_keys(self, simhash):
        for i, offset in enumerate(self.offsets):
            if i == (len(self.offsets) - 1):
                m = 2 ** (self.f - offset) - 1
            else:
                m = 2 ** (self.offsets[i + 1] - offset) - 1
            c = simhash.value >> offset & m
            yield '%x:%x' % (c, i)

    def bucket_size(self):
        return len(self.bucket)

In [16]:
# test data
data = {
    1: 'How are you? I Am fine. blar blar blar blar blar Thanks.',
    2: 'How are you i am fine. blar blar blar blar blar than',
    3: 'This is simhash test.',
    4: 'How are you i am fine. blar blar blar blar blar thank1',
}

objs = [(str(k), Simhash(v)) for k, v in data.items()]
index = SimhashIndex(objs, k=10)

OBJ_ID =  1 ; value =  8476273622247677034
KEYS =>  ['a:0', '3:1', '7:2', 'c:3', '11:4', '0:5', 'd:6', '1e:7', '5:8', 'e:9', '1d68:a']
	key =>  a:0 ; current set =>  set([])
	key =>  3:1 ; current set =>  set([])
	key =>  7:2 ; current set =>  set([])
	key =>  c:3 ; current set =>  set([])
	key =>  11:4 ; current set =>  set([])
	key =>  0:5 ; current set =>  set([])
	key =>  d:6 ; current set =>  set([])
	key =>  1e:7 ; current set =>  set([])
	key =>  5:8 ; current set =>  set([])
	key =>  e:9 ; current set =>  set([])
	key =>  1d68:a ; current set =>  set([])
OBJ_ID =  2 ; value =  8440240356449459322
KEYS =>  ['1a:0', '3:1', '7:2', 'c:3', '11:4', '4:5', 'b:6', '1c:7', '1:8', 'e:9', '1d48:a']
	key =>  1a:0 ; current set =>  set([])
	key =>  3:1 ; current set =>  set(['75a1c5f341161c6a,1'])
	key =>  7:2 ; current set =>  set(['75a1c5f341161c6a,1'])
	key =>  c:3 ; current set =>  set(['75a1c5f341161c6a,1'])
	key =>  11:4 ; current set =>  set(['75a1c5f341161c6a,1'])
	key =>  4:5 ; cur

In [12]:
# for k, v in index.bucket:
#     print(k, ' => ', ','.join(v))
index.bucket

defaultdict(set,
            {'0:5': {'75a1c5f341161c6a,1', 'f521c1f241169c6a,4'},
             '11:4': {'7521c1e2c9161c7a,2',
              '75a1c5f341161c6a,1',
              'f521c1f241169c6a,4'},
             '15:7': {'8a8fa4aeb77e08d7,3'},
             '17:0': {'8a8fa4aeb77e08d7,3'},
             '17:4': {'8a8fa4aeb77e08d7,3'},
             '1:8': {'7521c1e2c9161c7a,2', 'f521c1f241169c6a,4'},
             '1a:0': {'7521c1e2c9161c7a,2'},
             '1a:6': {'8a8fa4aeb77e08d7,3'},
             '1b:5': {'8a8fa4aeb77e08d7,3'},
             '1c:3': {'8a8fa4aeb77e08d7,3'},
             '1c:7': {'7521c1e2c9161c7a,2'},
             '1d48:a': {'7521c1e2c9161c7a,2'},
             '1d68:a': {'75a1c5f341161c6a,1'},
             '1d:9': {'8a8fa4aeb77e08d7,3'},
             '1e:7': {'75a1c5f341161c6a,1', 'f521c1f241169c6a,4'},
             '22a3:a': {'8a8fa4aeb77e08d7,3'},
             '2:2': {'8a8fa4aeb77e08d7,3'},
             '3:1': {'7521c1e2c9161c7a,2',
              '75a1c5f341161c6a,1'

In [7]:
for i, o in objs:
    print(o.value)
    print('%x,%s' % (o.value, i))
    print '-'*10

8476273622247677034
75a1c5f341161c6a,1
----------
8440240356449459322
7521c1e2c9161c7a,2
----------
9984379969213434071
8a8fa4aeb77e08d7,3
----------
17663612459742043242
f521c1f241169c6a,4
----------


In [6]:
s = Simhash('How are you? I Am fine. blar blar blar blar blar Thanks.')
s.value

8476273622247677034

In [17]:
s1 = Simhash(u'How are you i am fine.ablar ablar xyz blar blar blar blar blar blar blar thank')
dups = index.get_near_dups(s1)
dups

['1a:0', '3:1', '7:2', 'c:3', '11:4', '5:5', '9:6', '1e:7', '1:8', 'e:9', '3d08:a']
['e:9', '9:6', 'c:3', '3d48:a', '15:7', 'd:3', '1a:0', '2:2', '1d48:a', '4:8', '22a3:a', '1:8', '4:5', '1a:6', '0:5', '1c:3', '1b:5', '1d68:a', '1c:7', '5:8', '3:1', '1d:9', 'b:6', '11:4', '7:2', '17:4', '17:0', 'd:6', 'a:0', '6:1', '1e:7']
****************************************


['1', '2', '4']

In [15]:
{'0:5': {'75a1c5f341161c6a,1', 'f521c1f241169c6a,4'},
             '11:4': {'7521c1e2c9161c7a,2',
              '75a1c5f341161c6a,1',
              'f521c1f241169c6a,4'},
             '15:7': {'8a8fa4aeb77e08d7,3'},
             '17:0': {'8a8fa4aeb77e08d7,3'},
             '17:4': {'8a8fa4aeb77e08d7,3'},
             '1:8': {'7521c1e2c9161c7a,2', 'f521c1f241169c6a,4'},
             '1a:0': {'7521c1e2c9161c7a,2'},
             '1a:6': {'8a8fa4aeb77e08d7,3'},
             '1b:5': {'8a8fa4aeb77e08d7,3'},
             '1c:3': {'8a8fa4aeb77e08d7,3'},
             '1c:7': {'7521c1e2c9161c7a,2'},
             '1d48:a': {'7521c1e2c9161c7a,2'},
             '1d68:a': {'75a1c5f341161c6a,1'},
             '1d:9': {'8a8fa4aeb77e08d7,3'},
             '1e:7': {'75a1c5f341161c6a,1', 'f521c1f241169c6a,4'},
             '22a3:a': {'8a8fa4aeb77e08d7,3'},
             '2:2': {'8a8fa4aeb77e08d7,3'},
             '3:1': {'7521c1e2c9161c7a,2',
              '75a1c5f341161c6a,1',
              'f521c1f241169c6a,4'},
             '3d48:a': {'f521c1f241169c6a,4'},
             '4:5': {'7521c1e2c9161c7a,2'},
             '4:8': {'8a8fa4aeb77e08d7,3'},
             '5:8': {'75a1c5f341161c6a,1'},
             '6:1': {'8a8fa4aeb77e08d7,3'},
             '7:2': {'7521c1e2c9161c7a,2',
              '75a1c5f341161c6a,1',
              'f521c1f241169c6a,4'},
             '9:6': {'f521c1f241169c6a,4'},
             'a:0': {'75a1c5f341161c6a,1', 'f521c1f241169c6a,4'},
             'b:6': {'7521c1e2c9161c7a,2'},
             'c:3': {'7521c1e2c9161c7a,2', '75a1c5f341161c6a,1'},
             'd:3': {'f521c1f241169c6a,4'},
             'd:6': {'75a1c5f341161c6a,1'},
             'e:9': {'7521c1e2c9161c7a,2',
              '75a1c5f341161c6a,1',
              'f521c1f241169c6a,4'}}

{'0:5': {'75a1c5f341161c6a,1', 'f521c1f241169c6a,4'},
 '11:4': {'7521c1e2c9161c7a,2', '75a1c5f341161c6a,1', 'f521c1f241169c6a,4'},
 '15:7': {'8a8fa4aeb77e08d7,3'},
 '17:0': {'8a8fa4aeb77e08d7,3'},
 '17:4': {'8a8fa4aeb77e08d7,3'},
 '1:8': {'7521c1e2c9161c7a,2', 'f521c1f241169c6a,4'},
 '1a:0': {'7521c1e2c9161c7a,2'},
 '1a:6': {'8a8fa4aeb77e08d7,3'},
 '1b:5': {'8a8fa4aeb77e08d7,3'},
 '1c:3': {'8a8fa4aeb77e08d7,3'},
 '1c:7': {'7521c1e2c9161c7a,2'},
 '1d48:a': {'7521c1e2c9161c7a,2'},
 '1d68:a': {'75a1c5f341161c6a,1'},
 '1d:9': {'8a8fa4aeb77e08d7,3'},
 '1e:7': {'75a1c5f341161c6a,1', 'f521c1f241169c6a,4'},
 '22a3:a': {'8a8fa4aeb77e08d7,3'},
 '2:2': {'8a8fa4aeb77e08d7,3'},
 '3:1': {'7521c1e2c9161c7a,2', '75a1c5f341161c6a,1', 'f521c1f241169c6a,4'},
 '3d48:a': {'f521c1f241169c6a,4'},
 '4:5': {'7521c1e2c9161c7a,2'},
 '4:8': {'8a8fa4aeb77e08d7,3'},
 '5:8': {'75a1c5f341161c6a,1'},
 '6:1': {'8a8fa4aeb77e08d7,3'},
 '7:2': {'7521c1e2c9161c7a,2', '75a1c5f341161c6a,1', 'f521c1f241169c6a,4'},
 '9:6': {'f

In [7]:
s1 = Simhash("How are you i am fine.ablar ablar xyz blar blar blar blar blar blar blar thank", 64)
dups = index.get_near_dups(s1)

In [8]:
dups

['1', '2', '4']

In [18]:
z = collections.defaultdict(set)
z['a'] = {1, 2, 3}

In [19]:
z['b']

set()